# Challenge on predicting of resolvability of crimes for BATMAN


### Socio Team  or "Bat"Team 

<i>Goukam , Imad , Mohamed , Nassim (M2 AIC Paris_Sud) </i>

## Introduction
The data set was download in this url:<a>https://data.sfgov.org/Public-Safety/SFPD-Incidents-Current-Year-2015-/ritf-b9ki</a>. 
The data is thin, it contains

* Dates - timestamp of the crime incident
* Category - category of the crime incident (only in train.csv). This is the target variable you are going to predict.
* Descript - detailed description of the crime incident (only in train.csv)
* DayOfWeek - the day of the week
* PdDistrict - name of the Police Department District
* Resolution - how the crime incident was resolved (only in train.csv)
* Address - the approximate street address of the crime incident 
* X - Longitude
* Y - Latitude




** The goal is to predict the <code>Resolution</code> column. The prediction quality is measured by RMSE**. 



In [84]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Fetch the data and load it in pandas

In [85]:
import os

In [86]:
local_filename = 'data/train.csv'
data = pd.read_csv(local_filename, index_col=0)

In [87]:
data.shape

(153618, 11)

In [88]:
data.columns.values


array(['Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Location'], dtype=object)

In [89]:
data.head()

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location
IncidntNum,,,,,,,,,,,
160000108,ASSAULT,BATTERY,Thursday,12/31/2015,23:58,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
166004914,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Thursday,12/31/2015,23:55,SOUTHERN,NONE,1100 Block of MISSION ST,-122.411626,37.778590,"(37.7785895740312, -122.411626152299)"
160000095,ASSAULT,INFLICT INJURY ON COHABITEE,Thursday,12/31/2015,23:54,INGLESIDE,"ARREST, BOOKED",0 Block of LELAND AV,-122.404263,37.711339,"(37.7113387848327, -122.404262861765)"
160038137,OTHER OFFENSES,VIOLATION OF RESTRAINING ORDER,Thursday,12/31/2015,23:51,INGLESIDE,NONE,200 Block of BLYTHDALE AV,-122.420557,37.710895,"(37.7108945814914, -122.420556751442)"
166002930,NON-CRIMINAL,LOST PROPERTY,Thursday,12/31/2015,23:50,CENTRAL,NONE,800 Block of POST ST,-122.415844,37.787402,"(37.7874017655636, -122.41584375719)"


In [90]:
print min(data['Date'])
print max(data['Date'])

01/01/2015
12/31/2015


In [91]:
data['PdDistrict'].unique()

array(['SOUTHERN', 'INGLESIDE', 'CENTRAL', 'BAYVIEW', 'PARK', 'NORTHERN',
       'MISSION', 'TENDERLOIN', 'TARAVAL', 'RICHMOND'], dtype=object)

In [92]:
data['Category'].unique()

array(['ASSAULT', 'VANDALISM', 'OTHER OFFENSES', 'NON-CRIMINAL',
       'LARCENY/THEFT', 'VEHICLE THEFT', 'BURGLARY', 'ROBBERY', 'WARRANTS',
       'SUSPICIOUS OCC', 'WEAPON LAWS', 'DRUNKENNESS', 'TRESPASS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'MISSING PERSON',
       'SECONDARY CODES', 'FRAUD', 'EMBEZZLEMENT',
       'SEX OFFENSES, FORCIBLE', 'BRIBERY', 'STOLEN PROPERTY',
       'DISORDERLY CONDUCT', 'ARSON', 'FAMILY OFFENSES', 'RUNAWAY',
       'DRIVING UNDER THE INFLUENCE', 'KIDNAPPING', 'PROSTITUTION',
       'SUICIDE', 'LIQUOR LAWS', 'EXTORTION', 'GAMBLING', 'BAD CHECKS',
       'SEX OFFENSES, NON FORCIBLE', 'LOITERING',
       'PORNOGRAPHY/OBSCENE MAT', 'TREA'], dtype=object)

In [93]:
data.head()

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location
IncidntNum,,,,,,,,,,,
160000108,ASSAULT,BATTERY,Thursday,12/31/2015,23:58,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
166004914,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Thursday,12/31/2015,23:55,SOUTHERN,NONE,1100 Block of MISSION ST,-122.411626,37.778590,"(37.7785895740312, -122.411626152299)"
160000095,ASSAULT,INFLICT INJURY ON COHABITEE,Thursday,12/31/2015,23:54,INGLESIDE,"ARREST, BOOKED",0 Block of LELAND AV,-122.404263,37.711339,"(37.7113387848327, -122.404262861765)"
160038137,OTHER OFFENSES,VIOLATION OF RESTRAINING ORDER,Thursday,12/31/2015,23:51,INGLESIDE,NONE,200 Block of BLYTHDALE AV,-122.420557,37.710895,"(37.7108945814914, -122.420556751442)"
166002930,NON-CRIMINAL,LOST PROPERTY,Thursday,12/31/2015,23:50,CENTRAL,NONE,800 Block of POST ST,-122.415844,37.787402,"(37.7874017655636, -122.41584375719)"


In [94]:
data['Resolution'].unique()

array(['ARREST, BOOKED', 'NONE', 'UNFOUNDED', 'ARREST, CITED',
       'JUVENILE BOOKED', 'EXCEPTIONAL CLEARANCE', 'PSYCHOPATHIC CASE',
       'NOT PROSECUTED', 'LOCATED',
       'CLEARED-CONTACT JUVENILE FOR MORE INFO', 'JUVENILE DIVERTED',
       'JUVENILE CITED', 'COMPLAINANT REFUSES TO PROSECUTE',
       'JUVENILE ADMONISHED'], dtype=object)

In [95]:
data = data.replace(to_replace="ARREST, BOOKED", value=1)
data = data.replace(to_replace="ARREST, CITED", value=1)
data = data.replace(to_replace="NONE", value=0)
data = data.replace(to_replace="UNFOUNDED", value=0)
data = data.replace(to_replace="JUVENILE BOOKED", value=0)
data = data.replace(to_replace="EXCEPTIONAL CLEARANCE", value=0)
data = data.replace(to_replace="PSYCHOPATHIC CASE", value=0)
data = data.replace(to_replace="NOT PROSECUTED", value=0)
data = data.replace(to_replace="LOCATED", value=0)
data = data.replace(to_replace="CLEARED-CONTACT JUVENILE FOR MORE INFO", value=0)
data = data.replace(to_replace="COMPLAINANT REFUSES TO PROSECUTE", value=0)
data = data.replace(to_replace="JUVENILE ADMONISHED", value=0)
data = data.replace(to_replace="JUVENILE DIVERTED", value=0)
data = data.replace(to_replace="JUVENILE CITED", value=0)


*** Remove a "Address" To samplifier***

In [116]:
data = data.drop('Address', 1)


ValueError: labels ['Address'] not contained in axis

In [114]:
data.head()

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,X,Y,Location
IncidntNum,,,,,,,,,,
160000108,ASSAULT,BATTERY,Thursday,12/31/2015,23:58,Otisburg,1,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
166004914,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Thursday,12/31/2015,23:55,Otisburg,0,-122.411626,37.778590,"(37.7785895740312, -122.411626152299)"
160000095,ASSAULT,INFLICT INJURY ON COHABITEE,Thursday,12/31/2015,23:54,Burnley,1,-122.404263,37.711339,"(37.7113387848327, -122.404262861765)"
160038137,OTHER OFFENSES,VIOLATION OF RESTRAINING ORDER,Thursday,12/31/2015,23:51,Burnley,0,-122.420557,37.710895,"(37.7108945814914, -122.420556751442)"
166002930,NON-CRIMINAL,LOST PROPERTY,Thursday,12/31/2015,23:50,East End,0,-122.415844,37.787402,"(37.7874017655636, -122.41584375719)"


In [98]:
data.dtypes

Category       object
Descript       object
DayOfWeek      object
Date           object
Time           object
PdDistrict     object
Resolution      int64
X             float64
Y             float64
Location       object
dtype: object

*** Replace a San f PdDistrict with Gotham city district *** (<a>https://en.wikipedia.org/wiki/Gotham_City</a>)

In [99]:
data['PdDistrict'].unique()

array(['SOUTHERN', 'INGLESIDE', 'CENTRAL', 'BAYVIEW', 'PARK', 'NORTHERN',
       'MISSION', 'TENDERLOIN', 'TARAVAL', 'RICHMOND'], dtype=object)

In [100]:
data = data.replace(to_replace="SOUTHERN", value="Otisburg")
data = data.replace(to_replace="INGLESIDE", value="Burnley")
data = data.replace(to_replace="CENTRAL", value="East End")
data = data.replace(to_replace="BAYVIEW", value="Old Gotham")
data = data.replace(to_replace="PARK", value="Robinson Park")
data = data.replace(to_replace="NORTHERN", value="Chinatown")
data = data.replace(to_replace="MISSION", value="Bristol County")
data = data.replace(to_replace="TENDERLOIN", value="The Bowery")
data = data.replace(to_replace="TARAVAL", value="Diamond")
data = data.replace(to_replace="RICHMOND", value="Falcone Penthouse")

In [101]:
data.head()

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,X,Y,Location
IncidntNum,,,,,,,,,,
160000108,ASSAULT,BATTERY,Thursday,12/31/2015,23:58,Otisburg,1,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
166004914,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Thursday,12/31/2015,23:55,Otisburg,0,-122.411626,37.778590,"(37.7785895740312, -122.411626152299)"
160000095,ASSAULT,INFLICT INJURY ON COHABITEE,Thursday,12/31/2015,23:54,Burnley,1,-122.404263,37.711339,"(37.7113387848327, -122.404262861765)"
160038137,OTHER OFFENSES,VIOLATION OF RESTRAINING ORDER,Thursday,12/31/2015,23:51,Burnley,0,-122.420557,37.710895,"(37.7108945814914, -122.420556751442)"
166002930,NON-CRIMINAL,LOST PROPERTY,Thursday,12/31/2015,23:50,East End,0,-122.415844,37.787402,"(37.7874017655636, -122.41584375719)"


# Save a new data 

In [117]:
data = data.reset_index()

In [118]:
data.to_csv('data/out.csv')

### A baseline


In [119]:
data_encoded = data


In [120]:
features = data_encoded.drop(['Resolution'], axis=1)
X_columns = data_encoded.columns.drop(['Resolution'])
X = features.values
y = data_encoded['Resolution'].values

In [121]:
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

In [122]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression(multi_class='ovr')